In [26]:
# Import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pyspark.sql import SparkSession

In [27]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("PySpark and Neural Network") \
    .getOrCreate()

In [28]:
# Load the dataset
dataset_path = "obesity_dataset.csv"
df = spark.read.csv(dataset_path, header = True, inferSchema = True)

## 1. Height as predictor of obesity and overweight

In [29]:
# Select columns
data = df[['Height', 'Obese/Overweight']]
data.head()

Row(Height=1.62, Obese/Overweight='No')

In [30]:
# Convert to pandas dataframe
pandas_df = data.select("Height", "Obese/Overweight").toPandas()
pandas_df.head()

,Height,Obese/Overweight
0,1.62,No
1,1.52,No
2,1.80,No
3,1.80,Yes
4,1.78,Yes


In [31]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df['Obese/Overweight'] = label_encoder.fit_transform(pandas_df['Obese/Overweight'])

In [32]:
# Define features (X) and target (y)
X = pandas_df[['Height']] 
y = pandas_df['Obese/Overweight']   

In [33]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [34]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [37]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7311 - loss: 0.6138 - val_accuracy: 0.7544 - val_loss: 0.5867
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - accuracy: 0.7335 - loss: 0.6041 - val_accuracy: 0.7544 - val_loss: 0.5766
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.7305 - loss: 0.5889 - val_accuracy: 0.7544 - val_loss: 0.5714
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.7288 - loss: 0.5851 - val_accuracy: 0.7544 - val_loss: 0.5664
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.7220 - loss: 0.5873 - val_accuracy: 0.7544 - val_loss: 0.5624
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - accuracy: 0.7363 - loss: 0.5808 - val_accuracy: 0.7544 - val_loss: 0.5607
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.7483 - loss: 0.5646 - val_accuracy: 0.7544 - val_loss: 0.5587
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 695us/step - accuracy: 0.7306 - loss: 0.5752 - val_accura

In [38]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - accuracy: 0.7515 - loss: 0.5562
Neural Network Accuracy: 0.7352246046066284


## 2. Age as predictor of obesity/overweight

In [39]:
# Select columns
data2 = df[['Age', 'Obese/Overweight']]
data2.head()

Row(Age=21.0, Obese/Overweight='No')

In [40]:
# Convert to pandas dataframe
pandas_df2 = data2.select("Age", "Obese/Overweight").toPandas()
pandas_df2.head()

,Age,Obese/Overweight
0,21.0,No
1,21.0,No
2,23.0,No
3,27.0,Yes
4,22.0,Yes


In [41]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df2['Obese/Overweight'] = label_encoder.fit_transform(pandas_df2['Obese/Overweight'])

In [42]:
# Define features (X) and target (y)
X = pandas_df2[['Age']]
y = pandas_df2['Obese/Overweight']   

In [43]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [44]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [45]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [47]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5450 - loss: 0.6916 - val_accuracy: 0.7544 - val_loss: 0.6187
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - accuracy: 0.7441 - loss: 0.6050 - val_accuracy: 0.7544 - val_loss: 0.5539
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - accuracy: 0.7154 - loss: 0.5671 - val_accuracy: 0.7544 - val_loss: 0.5197
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.7214 - loss: 0.5317 - val_accuracy: 0.7544 - val_loss: 0.5042
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - accuracy: 0.7266 - loss: 0.5101 - val_accuracy: 0.7544 - val_loss: 0.4984
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - accuracy: 0.7178 - loss: 0.5122 - val_accuracy: 0.7574 - val_loss: 0.4963
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.7264 - loss: 0.5306 - val_accuracy: 0.7544 - val_loss: 0.4960
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.7565 - loss: 0.4915 - val_accura

In [48]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.7953 - loss: 0.4342
Neural Network Accuracy: 0.7612293362617493


## 3. Meals per day as predictor of obesity/overweight

In [49]:
# Select columns
data3 = df[['Meals_Per_Day', 'Obese/Overweight']]
data3.head()

Row(Meals_Per_Day=3.0, Obese/Overweight='No')

In [50]:
# Convert to pandas dataframe
pandas_df3 = data3.select("Meals_Per_Day", "Obese/Overweight").toPandas()
pandas_df3.head()

,Meals_Per_Day,Obese/Overweight
0,3.0,No
1,3.0,No
2,3.0,No
3,3.0,Yes
4,1.0,Yes


In [51]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df3['Obese/Overweight'] = label_encoder.fit_transform(pandas_df3['Obese/Overweight'])

In [52]:
# Define features (X) and target (y)
X = pandas_df3[['Meals_Per_Day']]
y = pandas_df3['Obese/Overweight']   

In [53]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [54]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [55]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [57]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6110 - loss: 0.6877 - val_accuracy: 0.7544 - val_loss: 0.6355
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.7310 - loss: 0.6293 - val_accuracy: 0.7544 - val_loss: 0.6014
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.7047 - loss: 0.6179 - val_accuracy: 0.7544 - val_loss: 0.5769
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.7359 - loss: 0.5774 - val_accuracy: 0.7544 - val_loss: 0.5619
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.7456 - loss: 0.5550 - val_accuracy: 0.7544 - val_loss: 0.5515
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - accuracy: 0.7549 - loss: 0.5529 - val_accuracy: 0.7604 - val_loss: 0.5459
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.7578 - loss: 0.5424 - val_accuracy: 0.7604 - val_loss: 0.5424
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - accuracy: 0.7560 - loss: 0.5446 - val_accura

In [58]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.7942 - loss: 0.4914
Neural Network Accuracy: 0.7777777910232544


## 4. Frequency of physical activity as predictor of obesity/overweight

In [59]:
# Select columns
data4 = df[['Frequency_Physical_Activity', 'Obese/Overweight']]
data4.head()

Row(Frequency_Physical_Activity=0.0, Obese/Overweight='No')

In [60]:
# Convert to pandas dataframe
pandas_df4 = data4.select("Frequency_Physical_Activity", "Obese/Overweight").toPandas()
pandas_df4.head()

,Frequency_Physical_Activity,Obese/Overweight
0,0.0,No
1,3.0,No
2,2.0,No
3,2.0,Yes
4,0.0,Yes


In [61]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df4['Obese/Overweight'] = label_encoder.fit_transform(pandas_df4['Obese/Overweight'])

In [62]:
# Define features (X) and target (y)
X = pandas_df4[['Frequency_Physical_Activity']]
y = pandas_df4['Obese/Overweight']   

In [63]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [64]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [65]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [66]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [67]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5566 - loss: 0.6863 - val_accuracy: 0.7544 - val_loss: 0.6294
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - accuracy: 0.7270 - loss: 0.6244 - val_accuracy: 0.7544 - val_loss: 0.5799
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.7166 - loss: 0.6050 - val_accuracy: 0.7544 - val_loss: 0.5637
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - accuracy: 0.7331 - loss: 0.5818 - val_accuracy: 0.7544 - val_loss: 0.5550
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - accuracy: 0.7343 - loss: 0.5661 - val_accuracy: 0.7544 - val_loss: 0.5524
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - accuracy: 0.7433 - loss: 0.5651 - val_accuracy: 0.7544 - val_loss: 0.5508
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step - accuracy: 0.7190 - loss: 0.5817 - val_accuracy: 0.7544 - val_loss: 0.5495
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - accuracy: 0.7427 - loss: 0.5585 - val_accura

In [68]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - accuracy: 0.7276 - loss: 0.5438
Neural Network Accuracy: 0.7210401892662048


## 5. Water intake as predictor of obesity/overweight

In [69]:
# Select columns
data5 = df[['Water_Intake', 'Obese/Overweight']]
data5.head()

Row(Water_Intake=2.0, Obese/Overweight='No')

In [70]:
# Convert to pandas dataframe
pandas_df5 = data5.select("Water_Intake", "Obese/Overweight").toPandas()
pandas_df5.head()

,Water_Intake,Obese/Overweight
0,2.0,No
1,3.0,No
2,2.0,No
3,2.0,Yes
4,2.0,Yes


In [71]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df5['Obese/Overweight'] = label_encoder.fit_transform(pandas_df5['Obese/Overweight'])

In [72]:
# Define features (X) and target (y)
X = pandas_df5[['Water_Intake']]
y = pandas_df5['Obese/Overweight']   

In [73]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [74]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [75]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [76]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [77]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5912 - loss: 0.6845 - val_accuracy: 0.7544 - val_loss: 0.6198
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.7446 - loss: 0.6105 - val_accuracy: 0.7544 - val_loss: 0.5688
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - accuracy: 0.7446 - loss: 0.5763 - val_accuracy: 0.7544 - val_loss: 0.5524
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.7250 - loss: 0.5853 - val_accuracy: 0.7544 - val_loss: 0.5477
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - accuracy: 0.7425 - loss: 0.5711 - val_accuracy: 0.7544 - val_loss: 0.5438
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - accuracy: 0.7217 - loss: 0.5811 - val_accuracy: 0.7544 - val_loss: 0.5430
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - accuracy: 0.7272 - loss: 0.5743 - val_accuracy: 0.7544 - val_loss: 0.5420
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.7266 - loss: 0.5812 - val_accura

In [78]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.7515 - loss: 0.4861
Neural Network Accuracy: 0.7352246046066284


## 6. Vegetable intake as predictor of obesity/overweight

In [79]:
# Select columns
data6 = df[['Vegetable_Intake', 'Obese/Overweight']]
data6.head()

Row(Vegetable_Intake=2.0, Obese/Overweight='No')

In [80]:
# Convert to pandas dataframe
pandas_df6 = data6.select("Vegetable_Intake", "Obese/Overweight").toPandas()
pandas_df6.head()

,Vegetable_Intake,Obese/Overweight
0,2.0,No
1,3.0,No
2,2.0,No
3,3.0,Yes
4,2.0,Yes


In [81]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df6['Obese/Overweight'] = label_encoder.fit_transform(pandas_df6['Obese/Overweight'])

In [82]:
# Define features (X) and target (y)
X = pandas_df6[['Vegetable_Intake']]
y = pandas_df6['Obese/Overweight']   

In [83]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [84]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [85]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [86]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [87]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7288 - loss: 0.6474 - val_accuracy: 0.7544 - val_loss: 0.5943
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step - accuracy: 0.7390 - loss: 0.6028 - val_accuracy: 0.7544 - val_loss: 0.5804
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - accuracy: 0.7380 - loss: 0.5992 - val_accuracy: 0.7544 - val_loss: 0.5748
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.7213 - loss: 0.6046 - val_accuracy: 0.7544 - val_loss: 0.5693
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.7297 - loss: 0.5914 - val_accuracy: 0.7544 - val_loss: 0.5663
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - accuracy: 0.7295 - loss: 0.5894 - val_accuracy: 0.7544 - val_loss: 0.5629
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - accuracy: 0.7412 - loss: 0.5737 - val_accuracy: 0.7544 - val_loss: 0.5610
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - accuracy: 0.7420 - loss: 0.5697 - val_accura

In [88]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.7515 - loss: 0.5615
Neural Network Accuracy: 0.7352246046066284


## 7. Technology use as predictor of obesity/overweight

In [89]:
# Select columns
data7 = df[['Technology_Use', 'Obese/Overweight']]
data7.head()

Row(Technology_Use=1.0, Obese/Overweight='No')

In [90]:
# Convert to pandas dataframe
pandas_df7 = data7.select("Technology_Use", "Obese/Overweight").toPandas()
pandas_df7.head()

,Technology_Use,Obese/Overweight
0,1.0,No
1,0.0,No
2,1.0,No
3,0.0,Yes
4,0.0,Yes


In [91]:
# Encode categorical data
label_encoder = LabelEncoder()
pandas_df7['Obese/Overweight'] = label_encoder.fit_transform(pandas_df7['Obese/Overweight'])

In [92]:
# Define features (X) and target (y)
X = pandas_df7[['Technology_Use']]
y = pandas_df7['Obese/Overweight']   

In [93]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [94]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [95]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [96]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [97]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6270 - loss: 0.6790 - val_accuracy: 0.7544 - val_loss: 0.6322
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - accuracy: 0.7388 - loss: 0.6285 - val_accuracy: 0.7544 - val_loss: 0.6010
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.7368 - loss: 0.6027 - val_accuracy: 0.7544 - val_loss: 0.5841
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - accuracy: 0.7124 - loss: 0.6097 - val_accuracy: 0.7544 - val_loss: 0.5739
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - accuracy: 0.7324 - loss: 0.5839 - val_accuracy: 0.7544 - val_loss: 0.5644
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 0.7218 - loss: 0.5819 - val_accuracy: 0.7544 - val_loss: 0.5580
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.7465 - loss: 0.5557 - val_accuracy: 0.7544 - val_loss: 0.5556
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - accuracy: 0.7270 - loss: 0.5715 - val_accura

In [98]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.7515 - loss: 0.5613
Neural Network Accuracy: 0.7352246046066284
